<a href="https://colab.research.google.com/github/rahulkaundal333/LTE-PERFORMANCE-HISTOGRAM/blob/main/Utilization_check_Consolidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sites_database.xlsx to sites_database.xlsx


In [ ]:
# Load data
df = pd.read_excel('sites_database.xlsx')

# Create base map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)


In [ ]:
# Define color scale
def get_color(utilization):
    if utilization < 50:
        return 'green'
    elif 50 <= utilization < 70:
        return 'orange'
    else:
        return 'red'

# Define shape based on category
def get_shape(category):
    shapes = {
        'Airtel Yes': 'circle',
        'Airtel No': 'triangle',
        'Sure No': 'square',
        'Sure Yes': 'circle'
    }
    return shapes.get(category, 'circle')

In [ ]:
# Create feature groups per category
categories = df['Category'].unique()
category_groups = {cat: folium.FeatureGroup(name=cat) for cat in categories}

# Initialize MarkerCluster before the loop
marker_cluster = MarkerCluster().add_to(m)  # Initialize and add to the map

# Add markers
for _, row in df.iterrows():
    color = get_color(row['Utilization'])
    shape = get_shape(row['Category'])
    popup = f"Site: {row['Site Name']}<br>Utilization: {row['Utilization']}%"

    if shape == 'triangle':
        marker = folium.RegularPolygonMarker(
            location=[row['Latitude'], row['Longitude']],
            number_of_sides=3,
            radius=10,
            popup=popup,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )
    elif shape == 'square':
        marker = folium.RegularPolygonMarker(
            location=[row['Latitude'], row['Longitude']],
            number_of_sides=4,
            radius=10,
            rotation=45,
            popup=popup,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )
    else:  # default circle
        marker = folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=8,
            popup=popup,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        )

    marker.add_to(category_groups[row['Category']])



    # Inside the loop where you're adding markers:
for _, row in df.iterrows():
    color = get_color(row['Utilization'])

    site_name = row['Site Name']
    utilization = row['Utilization']

    popup = folium.Popup(f"<b>{site_name}</b><br>Utilization: {utilization}%", max_width=200)
    tooltip = site_name  # Shows on hover

    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=8,
        popup=popup,
        tooltip=tooltip,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
    ).add_to(marker_cluster)  # Now marker_cluster is defined

# Add groups to map
for group in category_groups.values():
    group.add_to(m)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Save map
m.save('sites_utilization_map.html')
print("Map saved as 'sites_utilization_map.html'")

Map saved as 'sites_utilization_map.html'


In [ ]:
display(m)